In [1]:
!pip install arxiv

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki.name

'wikipedia'

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vector_db=FAISS.from_documents(documents,OllamaEmbeddings())
retriever=vector_db.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002B67886C8B0>)

In [6]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [7]:
retriever_tool.name

'langsmith_search'

In [8]:
## Arxiv Tool
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [9]:
tools=[wiki,arxiv,retriever_tool]

In [10]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Projects\\Langchain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002B648390A60>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community

In [11]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [12]:
## Agents
from langchain.agents import AgentType, initialize_agent
from langchain.llms import Ollama
from langchain.memory import ConversationBufferMemory
llm = Ollama(
             model="llama2", 
             )

agent = initialize_agent(llm=llm,
                         tools=tools,
                         prompt=prompt,
                         agent = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                         verbose=True,
                         output_key = "result",
                         handle_parsing_errors = True,
                         max_iterations=3,
                         early_stopping_method="generate",
                         memory = ConversationBufferMemory(memory_key = 'chat_history')                    
                        )
# agent = initialize_agent(llm=llm,
#                          tools=tools,
#                          prompt=prompt,
#                          verbose=True,
#                          max_iterations=3               
#                         )



d:\Projects\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [13]:
# ## Agent Executer
# from langchain.agents import AgentExecutor
# agent_executor=AgentExecutor(
#         agent=agent,
#         tools=tools,
#         verbose=True,
#         handle_parsing_errors=True,
#         return_intermediate_steps=True,
# )
# agent_executor

In [14]:
# agent_executor.invoke({"input":"Tell me about Langsmith"})

In [15]:

agent.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: langsmith_search
Action Input: Langsmith
Observation: Getting started with LangSmith | 🦜️🛠️ LangSmith

team deals with sensitive data that cannot be logged. How can I ensure that only my team can access it?​If you are interested in a private deployment of LangSmith or if you need to self-host, please reach out to us at sales@langchain.dev. Self-hosting LangSmith requires an annual enterprise license that also comes with support and formalized access to the LangChain team.Was this page helpful?NextUser GuideIntroductionInstall LangSmithCreate an API keySetup your environmentLog your first traceCreate your first evaluationNext StepsAdditional ResourcesFAQHow do I migrate projects between organizations?Why aren't my runs aren't showing up in my project?My team deals with sensitive data that cannot be logged. How can I ensure that only my team can access it?CommunityDiscordTwitterGitHubDocs CodeLangSmith 

OutputParserException: Could not parse LLM output: `Action: Create an API key for LangSmith.

To use LangSmith, you will need to create an API key. You can do this by running the following command in your terminal or command prompt:
```
langsmith api-key create
```
This will generate a new API key for you, which you can use to authenticate your requests to the LangSmith server. Once you have created your API key, you can use it to log traces and evaluate LLMs using LangSmith.

Next Steps:

1. Log your first trace with the generated API key. To do this, run the following command in your terminal or command prompt:
```
langsmith trace my_first_trace --api-key <YOUR_API_KEY>
```
2. Create an evaluation using the generated API key. To do this, run the following command in your terminal or command prompt:
```
langsmith evaluate my_first_evaluation --api-key <YOUR_API_KEY>
```
3. Explore additional LangSmith features and capabilities, such as proxying, tracing, and the Prompt Hub.
4. Join th

In [16]:
agent.invoke({"input":"Tell me about Machine Learning"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: arxiv
Action Input: What are the latest developments in machine learning?
Observation: Published: 2021-04-12
Title: LearningCity: Knowledge Generation for Smart Cities
Authors: Dimitrios Amaxilatis, Georgios Mylonas, Evangelos Theodoridis, Luis Diez, Katerina Deligiannidou
Summary: Alth
Thought:AI: Sure! Machine learning is a rapidly growing field with many exciting developments. One of the latest trends is the use of deep learning techniques for tasks such as image and speech recognition, natural language processing, and predictive modeling. Another area of research is the development of new algorithms and architectures that can handle large amounts of data and provide more accurate predictions. Additionally, there is a growing interest in explainable AI, which aims to provide insights into how machine learning models make decisions and how they can be improved.

To learn more about the latest develo

{'input': 'Tell me about Machine Learning',
 'chat_history': '',
 'output': 'Yes, there are many other exciting developments in machine learning! For example, there is growing interest in transfer learning, which involves using pre-trained models as a starting point for new tasks. This can help reduce the amount of training data required and improve the accuracy of the model. Another area of research is in developing new architectures that can handle complex tasks such as multi-modal processing, which involves analyzing and integrating multiple types of data.\n\nThere are also many exciting applications of machine learning, such as natural language processing for chatbots and virtual assistants, computer vision for self-driving cars and medical imaging, and predictive modeling for supply chain management and financial forecasting. The field is constantly evolving, so there are always new developments to explore!'}

In [17]:
agent.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: arxiv
Action Input: 1605.08386
Observation: Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allo
Thought:AI: According to the paper "1605.08386" published in 2016, the authors Caprice Stanley and Tobias Windisch study graphs on lattice points and their edges that come from a finite set of Markov bases. The paper focuses on the heat-bath random walks on these graphs and provides a summary of its findings. If you need more information or clarification on this topic, feel free to ask!

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'chat_history': 'Human: Tell me about Machine Learning\nAI: Yes, there are many other exciting developments in machine learning! For example, there is growing interest in transfer learning, which involves using pre-trained models as a starting point for new tasks. This can help reduce the amount of training data required and improve the accuracy of the model. Another area of research is in developing new architectures that can handle complex tasks such as multi-modal processing, which involves analyzing and integrating multiple types of data.\n\nThere are also many exciting applications of machine learning, such as natural language processing for chatbots and virtual assistants, computer vision for self-driving cars and medical imaging, and predictive modeling for supply chain management and financial forecasting. The field is constantly evolving, so there are always new developments to explore!',
 'output': 'According to the paper "1605